In [1]:
from models import AudioEmbeddingNet

In [2]:
inp = './tmp/examples/utt0.wav'

In [3]:
model = AudioEmbeddingNet()

Using cache found in /Users/siddhantpathak/.cache/torch/hub/harritaylor_torchvggish_master


In [4]:
out = model(inp)

In [5]:
out.shape

torch.Size([1, 384])

In [6]:
out

tensor([[154.5000,   8.0000, 197.0000, 132.5000, 215.0000,  91.0000, 128.0000,
          83.0000, 129.5000, 222.5000,  83.5000,  18.5000, 123.5000, 163.5000,
          96.5000,  54.5000, 157.0000, 169.5000, 166.5000, 146.0000,  87.5000,
         238.0000, 134.5000, 138.0000,   0.0000, 225.0000,  73.5000,  70.0000,
          66.5000, 109.5000, 123.0000,  87.5000, 161.0000, 135.5000, 138.0000,
         126.5000, 111.0000,  39.0000,  91.0000,  96.0000, 141.0000, 126.5000,
           0.0000, 187.0000, 125.5000,  90.5000, 125.0000, 145.0000,  29.5000,
         141.0000, 198.0000, 114.0000, 108.5000, 123.0000, 130.0000,  97.0000,
          53.5000,  60.5000, 195.0000, 243.5000, 109.5000,   0.5000,  10.5000,
         182.0000, 223.5000, 127.5000, 236.5000, 209.5000, 162.0000, 138.5000,
         102.5000,  78.5000, 161.5000, 199.0000, 115.5000,  63.5000,  91.5000,
         255.0000,  55.5000, 255.0000,  87.0000,  80.5000, 157.0000, 230.5000,
         206.5000, 123.0000, 161.5000, 194.5000, 255

In [7]:
from torch import nn
import torch

class NNCell(nn.Module):
    def __init__(self, D_u, D_c, D_s, D_i, D_e, D_a):
        """
        D_u: Input dimension u
        D_c: CumulativeContext dimension
        D_s: SelfSpeaker dimension
        D_i: IntraSpeaker dimension
        D_e: EmotionState dimension
        D_a: Attention vector dimension
        """
        super(NNCell, self).__init__()

        # Defining the GRU cells for different states
        self.GRU_c  = nn.GRUCell(D_c + D_s + D_i + D_u, D_c)
        self.GRU_sk = nn.GRUCell(D_c + D_u, D_s)
        self.GRU_i  = nn.GRUCell(D_a + D_u, D_i)
        self.GRU_e  = nn.GRUCell(D_c + D_s + D_u, D_e)
        
    def forward(self, u, a, C_prev, S_prev, I_prev, E_prev):
        """
        u: Current input
        a: Attention vector
        C_prev: Previous CumulativeContext
        S_prev: Previous SelfSpeaker
        I_prev: Previous IntraSpeaker
        E_prev: Previous EmotionState
        """
        
        # Updating CumulativeContext (C)
        C_input = torch.cat([C_prev, S_prev, I_prev, u], dim=1)
        C_t = self.GRU_c(C_input, C_prev)
        
        # Updating SelfSpeaker (S)
        S_input = torch.cat([C_t, u], dim=1)
        S_t = self.GRU_sk(S_input, S_prev)
        
        # Updating IntraSpeaker (I)
        I_input = torch.cat([a, u], dim=1)
        I_t = self.GRU_i(I_input, I_prev)
        
        # Updating EmotionState (E)
        E_input = torch.cat([C_t, S_t, u], dim=1)
        E_t = self.GRU_e(E_input, E_prev)

        return C_t, S_t, I_t, E_t

In [8]:
cell = NNCell(D_u = 128, D_c = 128, D_s = 128, D_i = 128, D_e = 128, D_a = 128)

In [9]:
cell(out)

TypeError: NNCell.forward() missing 5 required positional arguments: 'a', 'C_prev', 'S_prev', 'I_prev', and 'E_prev'